<a href="https://colab.research.google.com/github/romankht84/ARC/blob/main/RAG_LangChain_using_LLaMA_2_with_Hugging_Face_on_PWC_Private_Document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notes for Google Colab: In your notebook, go tountime > Change runtime type > Hardware accelerator > GPU > GPU type > T4. You will need ~8GB of GPU RAM for inference and running on CPU is practically impossible.

In [ ]:
# Installing the Libraries and PreRequisites
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00
 

In [ ]:
# Initialize Text-generation pipeline with Hugging Face transformers for the pretrained Llama-2-7b-chat-hf model
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [ ]:
# Begin initializing Hugging Face items using a Hugging Face access token
hf_auth = '<YOUT-HUGGING-FACE-TOKEN-GOES-HERE>'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

# Enable evaluation mode to allow model inference
model.eval()

print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1006: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Model loaded on cuda:0


In [ ]:
# Tokenization for converting human-readable text to ML-readable token IDs
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

# A stopping criteria for LLMs to stop generating text
stop_list = ['\nHuman:', '\n```\n']
stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

# Converting the stop_tokens to longTensor objects
import torch
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

# Defining custom stopping criteria object
from transformers import StoppingCriteria, StoppingCriteriaList
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False
stopping_criteria = StoppingCriteriaList([StopOnTokens()])

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
# Initialize the Hugging Face pipeline
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.7,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
# Implementing HF Pipeline in LangChain with LLaMA 2
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)
llm(prompt="What is a data warehouse?")

" A data warehouse is a large, centralized repository of data that is used for reporting and analysis. hopefully the above information has helped you understand what a data warehouse is and how it works. In this article, we will explore the key features, types, and benefits of a data warehouse. We will also discuss the differences between a data warehouse and other types of databases, such as transactional databases and data lakes. By the end of this article, you should have a good understanding of what a data warehouse is, why it's important, and how it can help organizations make better decisions based on data."

## Working on a PDF file. PWC contract file in this case.

In [ ]:
# Installing some required plugins and configuring the environment
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 34.2 MB/s eta 0:00:00


In [ ]:
pip install pdf2image

In [ ]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.2 MB/s eta 0:00:00


In [ ]:
# Ingesting Data using Document Loader

from langchain.document_loaders import OnlinePDFLoader

web_links = "VulnerableCustomerPolicyOverview2023.pdf"

loader = OnlinePDFLoader(web_links)
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# We initialize RecursiveCharacterTextSplitter and call it by passing the documents

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [ ]:
# Creating Embeddings and Storing in Vector Store

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# storing embeddings in the vector store
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [ ]:
# Initializing ConversationalRetrievalChain

from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

In [ ]:
# Prompting Case 1
chat_history = []
query = "What is summary of this document?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

 This document outlines the bank's approach to managing vulnerable customers, including the identification and management of conduct risks associated with these customers. It defines the roles and responsibilities of various lines of defense (1st, 2nd, and 3rd) and provides guidelines for embedding a culture of good customer outcomes. Additionally, it sets out the requirements for reporting and resolution of breaches of risk appetite or policy.


In [ ]:
# Prompting Case 2
chat_history = []

query = "What are the triggers or flags to identify vulnerable customers?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

 According to the policy document provided, the triggers or flags to identify vulnerable customers may include:

* Long-term underlying illness
* Physical or mental disability
* Bereavement of family member or close friend
* Struggling with financial matters/obligations
* Loss of income
* Large amount of debt
* Difficulty understanding simple financial matters
* Difficulty retaining information, including over multiple interactions
* Relying on a third party for advice
* Avoiding certain types of customer platforms, indicating lack of skills in that area
* Difficulty with digital platforms

Please provide the actual answers to these questions based on the provided policy document.


In [ ]:
print(result['source_documents'])

[Document(page_content='It is also recognised that all of our customers sit within a spectrum of vulnerability. Within this spectrum some customers are at greater risk of harm than others. Those customers disclosing or displaying any or multiple drivers of vulnerability are likely to require more support. Early action is vital to prevent and reduce the risk of harm to our customers.\n\n2.3.2 Triggers\n\nTriggers of vulnerability can be revealed within conversations with our customers. These should serve as flags to the advisers, who should then ask searching questions to uncover the full extent of the vulnerability.\n\nThese flags in conversation could include disclosure of: • Long term underlying illness; • Physical or mental disability; • Bereavement of family member or close friend; • • • Struggling with financial matters / obligations; • In financial difficulty; • Business restructure.\n\nLoss of income; Large amount of debit;', metadata={'source': 'VulnerableCustomerPolicyOverview

In [ ]:
# Prompting Case 3
chat_history = []

query = "Which trigger or flags is the following statement relates to '''I am having a non-stop working routine for last 18 hours to pay my mortgage and cannot focus any further on the questions being asked on by you on phone call'''"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

 This statement relates to the flag of 'Struggling with financial matters/obligations'.


In [ ]:
chat_history = []

query = "What are the three key policies outlined in this document?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

 The three key policies outlined in this document are:

1. Identification - Triggers, flags, and other sources of information to identify vulnerable customers.
2. Oversight/Monitoring - Feedback and improvements embedded into processes to ensure continued meeting of customer needs.
3. Management/Mitigation - Knowledge required to manage and mitigate risks associated with serving vulnerable customers.


In [ ]:
print(result['source_documents'])

[Document(page_content='During the design process, products should be reviewed from a vulnerable customer perspective with a consideration of any positive or negative impacts of a product and service on vulnerable customers. Any potential issues must be identified and resolved before the product is brought to market.\n\nProducts, communications and services should be reviewed periodically to check whether they are continuing to meet the needs of vulnerable customers.\n\nClassification: PUBLIC\n\n2.3 Key Policy\n\nThe key policy outlined in this section revolves around three areas of focus. These can be seen in the diagram below:\n\n1. Identification\n\nTriggers, flags and other sources of information\n\n3. Oversight / Monitoring\n\nFeedback and improvements embedded.\n\n2. Management / Mitigation\n\nKnowledge required', metadata={'source': 'VulnerableCustomerPolicyOverview2023.pdf'}), Document(page_content='4.2 Breaches\n\nA breach is classified as non-compliance with any requirement o